# Example of similarity score calculation of 2CO on Pt(553) configurations

### Import the necessary functions

In [1]:
import pandas as pd
import time # timing the execution of the similarity calculation
import similarity as sim
print("successfully loaded packages")

successfully loaded packages


### Load the configurations
There are 2 pairs of conifgurations (total of 4 configurations) subjected to the similarity calculation.\
The configurations in conf_pair1 (conf1 and conf2) are highly similar.\
The configurations in conf_pair2 (conf3 and conf4) are highly dis-similar.
conf1             |  conf2
:-------------------------:|:-------------------------:
![](conf_img/conf1.png)  |  ![](conf_img/conf2.png)

conf3             |  conf4
:-------------------------:|:-------------------------:
![](conf_img/conf3.png)  |  ![](conf_img/conf4.png)

### Note about loading the configuration

The configurations of interest can be placed arbitrarily. However, the path to these configurations \
must be accurate. The path can be either the absolute path, or the relative path w.r.t. this notebook. \
In this example, relative path is used for each configuration.

In [3]:
conf1_path = '2COconf1.CONTCAR'
conf2_path = '2COconf2.CONTCAR'
conf3_path = '2COconf3.CONTCAR'
conf4_path = '2COconf4.CONTCAR'

### Making configuration pairs
In this example, two configuration pairs are defined as previously stated. \
However, users are free to define their own configuration pairs, as long as \
each pair only consists two configurations.

If users want to do similarity calculation on all possible pairs among the configurations of interest, \
users should use `itertools.combinations`, and is demonstrated below.

In [4]:
conf_pair1 = [conf1_path,conf2_path]
conf_pair2 = [conf3_path,conf4_path]

### define all possible pair among the configurations of interest
from itertools import combinations # users probably need to install the itertools package.
conf_path_arr = [conf1_path, conf2_path, conf3_path, conf4_path]
all_conf_pairs = list(combinations(conf_path_arr,2))
print(all_conf_pairs)
print(len(all_conf_pairs)) # equal to N_conf C 2; number of ways to select 2 configurations among 4 configurations in total.

[('2COconf1.CONTCAR', '2COconf2.CONTCAR'), ('2COconf1.CONTCAR', '2COconf3.CONTCAR'), ('2COconf1.CONTCAR', '2COconf4.CONTCAR'), ('2COconf2.CONTCAR', '2COconf3.CONTCAR'), ('2COconf2.CONTCAR', '2COconf4.CONTCAR'), ('2COconf3.CONTCAR', '2COconf4.CONTCAR')]
6


### Do the similarity calculations
the main function to calculate the similarity score between two configurations of interest is `sim.compare_eigval_diff`. \
The two key inputs are:
* the configuration pair, which is an array/list/tuple of the paths of the two configurations.
* `start_atom_ele`, which sets element of the root atom for the Breadth-first search (BFS) \
for constructing the adjacency matrix of each chemical environment graph ($G_{chem-env}$). \
The sequence of atoms outputed from the BFS won't change the results, as we use the eigenvalues \
sorted by the magnitude.

It is highly recommended to use the surface atom element because surface atoms \
are always present in each $G_{chem-env}$. In this case, \
we use `Pt`, which is also the default setting.

In [5]:
start = time.time()
conf_pair1_score = sim.compare_eigval_diff(conf_pair1,start_atom_ele='Pt')
conf_pair2_score = sim.compare_eigval_diff(conf_pair2,start_atom_ele='Pt')
end = time.time()
t_execution = end - start

print(f'conf_pair1 score is {conf_pair1_score}')
print(f'conf_pair2 score is {conf_pair2_score}')
print(f'execution time is {t_execution} s')

similarity_arr = []
for pair in all_conf_pairs:
    score = sim.compare_eigval_diff(pair,start_atom_ele='Pt')
    similarity_arr.append(score)
    print(f'the sim score between {pair[0]} and {pair[1]} is {score}')

conf_pair1 score is -0.00010181489051319659
conf_pair2 score is -0.24061769247055054
execution time is 0.8043200969696045 s
the sim score between 2COconf1.CONTCAR and 2COconf2.CONTCAR is -0.00010181489051319659
the sim score between 2COconf1.CONTCAR and 2COconf3.CONTCAR is -0.18733686208724976
the sim score between 2COconf1.CONTCAR and 2COconf4.CONTCAR is -0.06794796139001846
the sim score between 2COconf2.CONTCAR and 2COconf3.CONTCAR is -0.1873398721218109
the sim score between 2COconf2.CONTCAR and 2COconf4.CONTCAR is -0.06794655323028564
the sim score between 2COconf3.CONTCAR and 2COconf4.CONTCAR is -0.24061769247055054


### Save the results
If users want to save the results, we recomment use the pandas package, and users can save the results into
a csv file.

In [6]:
conf0_path_arr = [i[0] for i in all_conf_pairs] # the array of the path to the configuration i in the configuration pairs
conf1_path_arr = [i[1] for i in all_conf_pairs] # the array of the path to the configuration j in the configuration pairs
[print(i) for i in all_conf_pairs]
print(conf0_path_arr)
print(conf1_path_arr)

('2COconf1.CONTCAR', '2COconf2.CONTCAR')
('2COconf1.CONTCAR', '2COconf3.CONTCAR')
('2COconf1.CONTCAR', '2COconf4.CONTCAR')
('2COconf2.CONTCAR', '2COconf3.CONTCAR')
('2COconf2.CONTCAR', '2COconf4.CONTCAR')
('2COconf3.CONTCAR', '2COconf4.CONTCAR')
['2COconf1.CONTCAR', '2COconf1.CONTCAR', '2COconf1.CONTCAR', '2COconf2.CONTCAR', '2COconf2.CONTCAR', '2COconf3.CONTCAR']
['2COconf2.CONTCAR', '2COconf3.CONTCAR', '2COconf4.CONTCAR', '2COconf3.CONTCAR', '2COconf4.CONTCAR', '2COconf4.CONTCAR']


In [7]:
sim_scores_path = f'2CO_similarity_scores.csv' # path to the result csv that users want to save data

### Define a dataframe with 3 col
results_contents_df = pd.DataFrame({'config1':conf0_path_arr,
                                    'config2':conf1_path_arr,
                                    'similarity':similarity_arr})
print(results_contents_df)                              

### save the similarity scores dataframe with scores from high (highly similar) to low (highly non-similar)
results_contents_df.sort_values('similarity').to_csv(sim_scores_path)

            config1           config2  similarity
0  2COconf1.CONTCAR  2COconf2.CONTCAR   -0.000102
1  2COconf1.CONTCAR  2COconf3.CONTCAR   -0.187337
2  2COconf1.CONTCAR  2COconf4.CONTCAR   -0.067948
3  2COconf2.CONTCAR  2COconf3.CONTCAR   -0.187340
4  2COconf2.CONTCAR  2COconf4.CONTCAR   -0.067947
5  2COconf3.CONTCAR  2COconf4.CONTCAR   -0.240618
